# Processing mixed beverage data
This Jupyter Notebook uses curl down download [Mixed Beverage Gross Receipts](https://comptroller.texas.gov/taxes/mixed-beverage/receipts.php) files from the Texas Comptroller's [data center](https://comptroller.texas.gov/transparency/open-data/search-datasets/), and then a python library called [agate](http://agate.readthedocs.io/) to clean and process that data for [stories similar to this one](http://www.mystatesman.com/business/austin-alcohol-sales-percent-february/Oo2txZUkuDlqBl0rU9O1lJ/) on monthly alcohol sales.

This first part uses bash (which talks to the macOS) and curl to download the file we need.

### Downloading the file

In [1]:
%%bash
## downloads the mixedbev file
## You have to set this URL based on the data center
curl -O https://comptroller.texas.gov/auto-data/odc/MIXEDBEV_03_2017.CSV

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2493k  100 2493k    0     0  2692k      0 --:--:-- --:--:-- --:--:-- 2860k



There is supposedly a way to call a file from a [remote url](http://agate-remote.readthedocs.io/en/0.2.0/) into agate, but I use bash above to curl the file and store it locally instead.

Next, we'll use a bash command to peek at the data, which we know is a mess:

In [2]:
%%bash
head -n 5 MIXEDBEV_03_2017.CSV

"MB821424    ","ABI-HAUS                      ","959 N 2ND ST                  ","ABILENE             ","TX","79601","221","          ","2017/01", 000000523.40
"MB638028    ","ABILENE BEEHIVE INC           ","442 CEDAR ST STE A            ","ABILENE             ","TX","79601","221","          ","2017/02", 000002610.52
"MB543114    ","ABILENE BOWLING LANES INC     ","279 RUIDOSA AVE               ","ABILENE             ","TX","79605","221","          ","2017/02", 000000256.27
"MB933130    ","ABILENE CABARET LLC           ","1918 BUTTERNUT ST             ","ABILENE             ","TX","79602","221","          ","2017/02", 000000699.41
"N 037863    ","ABILENE COUNTRY CLUB          ","4039 S TREADAWAY BLVD         ","ABILENE             ","TX","79602","221","          ","2017/02", 000001801.63


Now that we have our file and know what it looks like, we'll use Python and the agate library to clean and analyze it. You'll need to make sure that you have agate installed, preferably in an virtual environment like Conda, as described in the [ReadMe](README.md).

In [3]:
# imports the libraries we will use
import agate
from decimal import Decimal
import re

/Users/christian/anaconda/envs/mixbev/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-04-03 23:31:15.339295. Please add timezone info to timestamps.


In [4]:
# this surpresses the timezone warning
# Might comment out during development so other warnings
# are not surpressed
import warnings
warnings.filterwarnings('ignore')

### Study variables

This is where you set which file you are working with, and which month you want to study, etc.

First, we'll list the files in our directory that we have downloaded so far so we can get the filename:

In [5]:
ls

MIXEDBEV_02_2017.CSV         counties.csv
MIXEDBEV_03_2017.CSV         headers.txt
Mixed beverages agate.ipynb  mixbev-env.txt
README.md


Then we set some values based on those.

- The **`file`** is the name of the file we want to process
- The **`tax_rate`** is the value we need for this file to get the Gross Recipts (vs the Tax Reported, which is just the tax amount the establishment paid). The comptroller [has information on the tax](https://comptroller.texas.gov/taxes/mixed-beverage/receipts.php), but this [old record layout](https://github.com/utdata/cli-tools/blob/master/data/mixbevtax/OLD-MIXEDBEVTAX-LAYOUT.txt) best describes the math.
- The **`month_studied`** is the YYYY/MM designation for the month before the file release. The file released in February has mostly records from January, but can also have any other month, so we set here the specific month we want. Note there is a check later on that counts the number of files by month, which is worth checking.

In [6]:
# this is our source file, which may have been downloaded above
file = 'MIXEDBEV_03_2017.CSV'

# Sets the tax rate to convert Report Tax to Gross Receipts
# It's 6.7 since January 1, 2014
tax_rate = Decimal('6.7')

# setting the month_studied var.
# This should be checked in the table below that counts records by month
month_studied = '2017/02'


### Data variables
These probalby won't change between analysis or data sets.

In [7]:
# sets the column names of the original data set.
column_names = [
    'TABC Permit Number',
    'Trade Name',
    'Location Address',
    'Location City',
    'Location State',
    'Location Zip Code',
    'Location County Code',
    'Blank',
    'Report Period',
    'Report Tax'
]
# Helps us import some text fields that may be considered numbers in error.
specified_types = {
    'Location Zip Code': agate.Text(),
    'Location County Code': agate.Text()
}

### Import the file


In [8]:
# this imports the file specified above, along with the proper types
mixbev_raw = agate.Table.from_csv(file, column_names, encoding='iso-8859-1', column_types=specified_types)

# prints table fields so we an check thoes data types
print(mixbev_raw)

| column               | data_type |
| -------------------- | --------- |
| TABC Permit Number   | Text      |
| Trade Name           | Text      |
| Location Address     | Text      |
| Location City        | Text      |
| Location State       | Text      |
| Location Zip Code    | Text      |
| Location County Code | Text      |
| Blank                | Boolean   |
| Report Period        | Text      |
| Report Tax           | Number    |



### Clean up text fields and compute gross receipts

In [9]:
# mixbev_trim creates a new interim table with results of compute function
# that takes the four columns that need trimming and strips them of white space,
# adding them to the end of the table with new names.
# The last computation does the math to create the Gross Receipts based on the tax_rate set above

mixbev_trim = mixbev_raw.compute([
    ('Permit', agate.Formula(agate.Text(), lambda r: r['TABC Permit Number'].strip())),
    ('Name', agate.Formula(agate.Text(), lambda r: r['Trade Name'].strip())),
    ('Address', agate.Formula(agate.Text(), lambda r: r['Location Address'].strip())),
    ('City', agate.Formula(agate.Text(), lambda r: r['Location City'].strip())),
    ('Receipts_compute', agate.Formula(agate.Number(), lambda r: (r['Report Tax'] / tax_rate) * 100))
])

# the Receipts_compute computation above returns as a decimal number,
# so this function rounds those numbers.
# I might refactor this late so I can use it elsewhere.
def round_receipt(row):
    return row['Receipts_compute'].quantize(Decimal('0.01'))

# This compute method uses round_recipt function above,
# putting the results into a new table.
mixbev_round = mixbev_trim.compute([
    ('Receipts', agate.Formula(agate.Number(), round_receipt))
])

## shows the new columns added to the interim table
print(mixbev_round)

| column               | data_type |
| -------------------- | --------- |
| TABC Permit Number   | Text      |
| Trade Name           | Text      |
| Location Address     | Text      |
| Location City        | Text      |
| Location State       | Text      |
| Location Zip Code    | Text      |
| Location County Code | Text      |
| Blank                | Boolean   |
| Report Period        | Text      |
| Report Tax           | Number    |
| Permit               | Text      |
| Name                 | Text      |
| Address              | Text      |
| City                 | Text      |
| Receipts_compute     | Number    |
| Receipts             | Number    |



In [10]:
# creates new table, selecting just the columnse we need
# then renames some of them for ease.
mixbev_cleaned = mixbev_round.select([
    'Permit',
    'Name',
    'Address',
    'City',
    'Location State',
    'Location County Code',
    'Report Period',
    'Report Tax',
    'Receipts'
]).rename(column_names = {
    'Location State': 'State',
    'Location County Code': 'CountyCode',
    'Report Period': 'Period',
    'Report Tax': 'Tax'
})

## these are now the columns present in our new, cleaned table
print(mixbev_cleaned)

| column     | data_type |
| ---------- | --------- |
| Permit     | Text      |
| Name       | Text      |
| Address    | Text      |
| City       | Text      |
| State      | Text      |
| CountyCode | Text      |
| Period     | Text      |
| Tax        | Number    |
| Receipts   | Number    |



In [11]:
# and this peeks at a couple of columns the data (Tax and Receipts)
# to make sure they make sense and the math is right
# I did send this to_csv and made sure columns were trimmed, etc
mixbev_cleaned.select(['Tax','Receipts']).limit(5).print_table()

|      Tax |  Receipts |
| -------- | --------- |
| 2,610.52 | 38,962.99 |
|   256.27 |  3,824.93 |
|   699.41 | 10,438.96 |
| 1,801.63 | 26,890.00 |
| 1,782.66 | 26,606.87 |


### Create establishment column

We do this so we make sure we have single establishments instead of grouping trade names together from different addresses, like 'CHILI'S BAR & GRILL'.

In [12]:
# Concatenates the name and address
mixbev_cleaned_est = mixbev_cleaned.compute([
    ('Establishment', agate.Formula(agate.Text(), lambda r: '%(Name)s %(Address)s' % r))
])

# Prints columns so you see it is there
print(mixbev_cleaned_est)

| column        | data_type |
| ------------- | --------- |
| Permit        | Text      |
| Name          | Text      |
| Address       | Text      |
| City          | Text      |
| State         | Text      |
| CountyCode    | Text      |
| Period        | Text      |
| Tax           | Number    |
| Receipts      | Number    |
| Establishment | Text      |



In [13]:
# selects and prints Establishment to check what is looks like
mixbev_establishment = mixbev_cleaned_est.select('Establishment')
mixbev_establishment.print_table(max_column_width=80)

| Establishment                                      |
| -------------------------------------------------- |
| ABILENE BEEHIVE INC 442 CEDAR ST STE A             |
| ABILENE BOWLING LANES INC 279 RUIDOSA AVE          |
| ABILENE CABARET LLC 1918 BUTTERNUT ST              |
| ABILENE COUNTRY CLUB 4039 S TREADAWAY BLVD         |
| ABILENE SEAFOOD TAVERN 1882 S CLACK ST             |
| ABUELO'S BEVERAGE CORPORATION 4782 S 14TH ST       |
| ACE IN THE HOLE 133 EPLENS CT                      |
| AMNESIA, LLC. 1850 S CLACK ST                      |
| BILLIARDS PLUS 5495 S 7TH ST                       |
| BONZAI JAPANESE STEAK HOUSE 1802 S CLACK ST        |
| BREAKERS SPORTS BAR 1874 S CLACK ST                |
| BUFFALO WILD WINGS GRILL & BAR 1010 E OVERLAND TRL |
| BUFFALO WILD WINGS GRILL AND B 4401 RIDGEMONT DR   |
| CAHOOTS CATFISH & OYSTER BAR/J 301 S 11TH ST       |
| CHELSEA'S ST PUB 4310 BUFFALO GAP RD STE 1342      |
| CHILI'S GRILL & BAR 4302 S CLACK ST                |
| CHILIS G

### Import and merge counties lookup table
We do this to get county names. I got this list from the comptroller.

In [14]:
# importing countes.csv, ensuring that the 'code' column is text
counties = agate.Table.from_csv('counties.csv', column_types={'code': agate.Text()})

# peek at the column names
print(counties)

| column | data_type |
| ------ | --------- |
| id     | Number    |
| county | Text      |
| code   | Text      |



In [15]:
# peek at the data
counties.limit(5).print_table()

| id | county   | code |
| -- | -------- | ---- |
|  1 | Anderson | 001  |
|  2 | Andrews  | 002  |
|  3 | Angelina | 003  |
|  4 | Aransas  | 004  |
|  5 | Archer   | 005  |


In [16]:
# joines the counties table to the mixed bev cleaned data with establishments
mixbev_joined = mixbev_cleaned_est.join(counties, 'CountyCode', 'code')

# check that the merge was succesful 
print(mixbev_joined)

| column        | data_type |
| ------------- | --------- |
| Permit        | Text      |
| Name          | Text      |
| Address       | Text      |
| City          | Text      |
| State         | Text      |
| CountyCode    | Text      |
| Period        | Text      |
| Tax           | Number    |
| Receipts      | Number    |
| Establishment | Text      |
| id            | Number    |
| county        | Text      |



In [17]:
# get just the columns we need and rename county
# THIS is the finished, cleaned mixbev table
mixbev = mixbev_joined.select([
    'Permit',
    'Name',
    'Address',
    'Establishment',
    'City',
    'State',
    'county',
    'Period',
    'Tax',
    'Receipts'
]).rename(column_names = {
    'county': 'County'
})

# peek at the column names
print(mixbev)

| column        | data_type |
| ------------- | --------- |
| Permit        | Text      |
| Name          | Text      |
| Address       | Text      |
| Establishment | Text      |
| City          | Text      |
| State         | Text      |
| County        | Text      |
| Period        | Text      |
| Tax           | Number    |
| Receipts      | Number    |



In [18]:
# peek at the table
mixbev.limit(5).print_table()

| Permit   | Name                 | Address              | Establishment        | City    | State | ... |
| -------- | -------------------- | -------------------- | -------------------- | ------- | ----- | --- |
| MB638028 | ABILENE BEEHIVE INC  | 442 CEDAR ST STE A   | ABILENE BEEHIVE I... | ABILENE | TX    | ... |
| MB543114 | ABILENE BOWLING L... | 279 RUIDOSA AVE      | ABILENE BOWLING L... | ABILENE | TX    | ... |
| MB933130 | ABILENE CABARET LLC  | 1918 BUTTERNUT ST    | ABILENE CABARET L... | ABILENE | TX    | ... |
| N 037863 | ABILENE COUNTRY CLUB | 4039 S TREADAWAY ... | ABILENE COUNTRY C... | ABILENE | TX    | ... |
| MB200506 | ABILENE SEAFOOD T... | 1882 S CLACK ST      | ABILENE SEAFOOD T... | ABILENE | TX    | ... |


### Looking at dates of the records

Here we are looking at the entire mixbev table to see what range of dates we have. This way we can make sure we are analyzing the correct month based on this data. (More than one month can be present, but it will be predominately the previous month).

To explain what we are doing here, as it is kind of obtuse in agage:
- use group_by to create a tableset by the Period field.
- Create a table and set it it based on counting the number of records for each Period.
- create a table to then sort the period in reverse order to put the dominate month at the top
- Then print the sorted table (top 10 rows)

In [19]:
# this is the group_by
by_period = mixbev.group_by('Period')

# Then aggregate that group by count of records in Period
period_totals = by_period.aggregate([
    ('count', agate.Count())
])

# Take those results and sort them
period_totals_sorted = period_totals.order_by('count', reverse=True)

# prints the table of period and number of records
period_totals_sorted.print_table(max_rows=None)


| Period  |  count |
| ------- | ------ |
| 2017/02 | 14,090 |
| 2017/01 |  1,423 |
| 2016/12 |    141 |
| 2016/11 |     52 |
| 2017/03 |     32 |
| 2016/10 |     26 |
| 2016/09 |     21 |
| 2016/08 |     13 |
| 2016/07 |      9 |
| 2016/05 |      8 |
| 2016/06 |      8 |
| 2016/04 |      5 |
| 2016/03 |      4 |
| 2015/02 |      4 |
| 2016/02 |      3 |
| 2014/12 |      3 |
| 2014/11 |      3 |
| 2015/12 |      2 |
| 2009/01 |      2 |
| 2014/10 |      2 |
| 2015/01 |      2 |
| 2015/03 |      2 |
| 2015/11 |      1 |
| 2011/05 |      1 |
| 2011/10 |      1 |
| 2016/01 |      1 |
| 2014/07 |      1 |
| 2015/10 |      1 |


The top value in the table above is typically the month before the reporting date. This also shows how many records are filed for OTHER months. We want to make sure the top month value is included as the **month_studied** variable at the top of this file.

So, now we can filter the data to our specific month, which will use for the rest of the analysis:

In [20]:
## filters the records to our month_studied
mixbev_month = mixbev.where(lambda row: row['Period'] == month_studied)

## The number of records in our month
len(mixbev_month)

14090

## Top sales statewide

In [21]:
# groups the data based on Establishment and City
mixbev_grouped = mixbev_month.group_by('Establishment').group_by('County').group_by('City')

# computes the sales based on the grouping
state_summary = mixbev_grouped.aggregate([
    ('Sales_sum', agate.Sum('Receipts'))
])

# sorts the results by most sold
state_summary_sorted = state_summary.order_by('Sales_sum', reverse=True)

# prints the top 10 results
state_summary_sorted.limit(10).print_table(max_column_width=80)

| Establishment                                           | County  | City        |    Sales_sum |
| ------------------------------------------------------- | ------- | ----------- | ------------ |
| ARAMARK SPORTS AND ENTERTAINME 211 AT AND T CENTER PKWY | Bexar   | SAN ANTONIO | 2,889,877.91 |
| GAYLORD TEXAN 1501 GAYLORD TRL                          | Tarrant | GRAPEVINE   | 1,875,121.94 |
| SALC, INC. 2201 N STEMMONS FWY FL 1                     | Dallas  | DALLAS      | 1,384,787.91 |
| LEVY RESTAURANTS 2500 VICTORY AVE                       | Dallas  | DALLAS      | 1,254,393.88 |
| AT&T STADIUM 1 LEGENDS WAY                              | Tarrant | ARLINGTON   | 1,225,221.94 |
| WLS BEVERAGE CO 110 E 2ND ST                            | Travis  | AUSTIN      | 1,191,110.90 |
| HILTON AMERICAS - HOUSTON 1600 LAMAR ST                 | Harris  | HOUSTON     | 1,169,312.99 |
| HOSPITALITY INTERNATIONAL, INC 23808 RESORT PKWY        | Bexar   | SAN ANTONIO | 1,144,965.97 |
| LEVY RES

## Overall statewide sum

In [22]:
# summing sales statewide for month
mixbev_month.aggregate(agate.Sum('Receipts'))

Decimal('520914084.60')

## Location sums function

This function allows us to pass in a city or county name to filter the monthly receipts table and then sum the Tax and Receipts columns. The result can then be acted on to print or aggreggate. We do this as a function that we can reuse because we know we need to do it for bunch of cities or counties.

In [23]:
# function to group sales by a specific city
# City or County passed in should be ALL CAPS
# Location_type can be 'City' or 'County'

def location_sum(location_type, location):
    # Filters the data to the specified city
    location_filtered = mixbev_month.where(lambda row: row[location_type].upper() == location)

    # groups the data based on Establishment and location
    location_grouped = location_filtered.group_by('Establishment').group_by(location_type)
    # computes the sales based on the grouping
    location_summary = location_grouped.aggregate([
        ('Tax_sum', agate.Sum('Tax')),
        ('Receipts_sum', agate.Sum('Receipts'))
    ])
    # sorts the results by most sold
    location_summary_sorted = location_summary.order_by('Receipts_sum', reverse=True)
    # prints the top 10 results
    
    return(location_summary_sorted)


## Austin sales and sums

With this, we refernce the location_sum function above, and pass the type of location (City) and the name of the city (AUSTIN). At the same time, we limit the result of that function to the first 10 records, and then print the results. We are basically stringing together a bunch of stuff at once.

In [24]:
# uses the city_sum function to filter
location_sum('City', 'AUSTIN').limit(10).print_table(max_column_width=60)

| Establishment                                      | City   |   Tax_sum | Receipts_sum |
| -------------------------------------------------- | ------ | --------- | ------------ |
| WLS BEVERAGE CO 110 E 2ND ST                       | AUSTIN | 79,804.43 | 1,191,110.90 |
| ROSE ROOM/ 77 DEGREES 11500 ROCK ROSE AVE          | AUSTIN | 29,873.89 |   445,878.96 |
| 400 BAR/CUCARACHA/CHUPACABRA/J 400 E 6TH ST        | AUSTIN | 28,395.13 |   423,807.91 |
| THE DOGWOOD DOMAIN 11420 ROCK ROSE AVE STE 700     | AUSTIN | 27,372.64 |   408,546.87 |
| KUNG FU SALOON 11501 ROCK ROSE AVE STE 140         | AUSTIN | 23,468.15 |   350,270.90 |
| BARTON CREEK COUNTRY CLUB 8212 BARTON CLUB DR      | AUSTIN | 23,065.68 |   344,263.88 |
| TOP GOLF 2700 ESPERANZA XING                       | AUSTIN | 22,937.51 |   342,350.90 |
| SAN JACINTO BEVERAGE COMPANY L 98 SAN JACINTO BLVD | AUSTIN | 22,361.92 |   333,760.00 |
| ALAMO DRAFTHOUSE CINEMA 1120 S LAMAR BLVD          | AUSTIN | 22,324.66 |   333,203.88 |

Agate not only allow us to print tables, but we can also print a simple, text-based bar chart with the "print_bars" method.

In [25]:
location_sum('City', 'AUSTIN').limit(10).print_bars('Establishment', 'Receipts_sum', width=80)

Establishment                                      Receipts_sum
WLS BEVERAGE CO 110 E 2ND ST                       1,191,110.90 ▓░░░░░░░░░      
ROSE ROOM/ 77 DEGREES 11500 ROCK ROSE AVE            445,878.96 ▓░░░            
400 BAR/CUCARACHA/CHUPACABRA/J 400 E 6TH ST          423,807.91 ▓░░░            
THE DOGWOOD DOMAIN 11420 ROCK ROSE AVE STE 700       408,546.87 ▓░░░            
KUNG FU SALOON 11501 ROCK ROSE AVE STE 140           350,270.90 ▓░░░            
BARTON CREEK COUNTRY CLUB 8212 BARTON CLUB DR        344,263.88 ▓░░░            
TOP GOLF 2700 ESPERANZA XING                         342,350.90 ▓░░░            
SAN JACINTO BEVERAGE COMPANY L 98 SAN JACINTO BLVD   333,760.00 ▓░░░            
ALAMO DRAFTHOUSE CINEMA 1120 S LAMAR BLVD            333,203.88 ▓░░             
THE PALAZIO 501 E BEN WHITE BLVD                     317,540.90 ▓░░             
                                                                +--------------+
                                             

### Total sales Austin

In [26]:
# Austin total sales as s city
# This sums the grouped table, but it works
location_sum('City', 'AUSTIN').aggregate(agate.Sum('Receipts_sum'))

Decimal('55751374.85')

## More Central Texas cities

In [27]:
location_sum('City', 'BASTROP').limit(5).print_table(max_column_width=60)

| Establishment                              | City    |  Tax_sum | Receipts_sum |
| ------------------------------------------ | ------- | -------- | ------------ |
| OLD TOWN RESTURANT AND BAR/PIN 931 MAIN ST | BASTROP | 4,089.61 |    61,038.96 |
| CHILI'S GRILL & BAR 734 HIGHWAY 71 W       | BASTROP | 2,659.63 |    39,695.97 |
| NEIGHBOR'S 601 CHESTNUT ST UNIT C          | BASTROP | 1,849.13 |    27,598.96 |
| LA HACIENDA RESTAURANT 1800 WALNUT ST      | BASTROP | 1,570.01 |    23,432.99 |
| VERANDA 910 MAIN ST                        | BASTROP | 1,420.40 |    21,200.00 |


In [28]:
location_sum('City', 'BEE CAVE').limit(3).print_table(max_column_width=60)

| Establishment                                       | City     |  Tax_sum | Receipts_sum |
| --------------------------------------------------- | -------- | -------- | ------------ |
| WOODY TAVERN AND GRILL, INC. 12801 SHOPS PKWY # 100 | BEE CAVE | 5,130.59 |    76,575.97 |
| MAUDIE'S HILL COUNTRY, LLC 12506 SHOPS PKWY         | BEE CAVE | 4,982.45 |    74,364.93 |
| CAFE BLUE 12800 HILL COUNTRY BLVD STE               | BEE CAVE | 4,913.11 |    73,330.00 |


In [29]:
location_sum('City', 'BUDA').limit(3).print_table(max_column_width=60)

| Establishment                       | City |  Tax_sum | Receipts_sum |
| ----------------------------------- | ---- | -------- | ------------ |
| BUCKS BACKYARD 1750 S FM 1626       | BUDA | 8,278.98 |   123,566.87 |
| WILLIE'S JOINT 824 MAIN ST          | BUDA | 3,891.36 |    58,080.00 |
| BROOKLYN'S DOWN SOUTH 100 N MAIN ST | BUDA | 3,007.83 |    44,892.99 |


In [30]:
location_sum('City', 'CEDAR PARK').limit(3).print_table(max_column_width=60)

| Establishment                                          | City       |  Tax_sum | Receipts_sum |
| ------------------------------------------------------ | ---------- | -------- | ------------ |
| CHUY'S 4911 183A TOLL RD                               | CEDAR PARK | 6,960.83 |   103,892.99 |
| LUPE TORTILLA MEXICAN RESTAURA 4501 183A TOLL RD STE B | CEDAR PARK | 6,852.29 |   102,272.99 |
| BJ'S RESTAURANT & BREWERY 1001 E WHITESTONE BLVD       | CEDAR PARK | 6,217.66 |    92,800.90 |


In [31]:
location_sum('City', 'DRIPPING SPRINGS').limit(3).print_table(max_column_width=60)

| Establishment                                       | City             |  Tax_sum | Receipts_sum |
| --------------------------------------------------- | ---------------- | -------- | ------------ |
| DEEP EDDY DISTILLING CO 2250 E HIGHWAY 290          | DRIPPING SPRINGS | 4,043.11 |    60,344.93 |
| TRUDY'S FOUR STAR 13059 FOUR STAR BLVD              | DRIPPING SPRINGS | 3,532.91 |    52,730.00 |
| FLORES MEXICAN RESTAURANT 2440 E HIGHWAY 290 BLDG D | DRIPPING SPRINGS | 3,298.87 |    49,236.87 |


In [32]:
location_sum('City', 'GEORGETOWN').limit(3).print_table(max_column_width=60)

| Establishment                            | City       |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ---------- | -------- | ------------ |
| GATEWAY BEVERAGES, INC 1101 WOODLAWN AVE | GEORGETOWN | 5,975.06 |    89,180.00 |
| EL MONUMENTO 205 W 2ND ST                | GEORGETOWN | 5,384.79 |    80,370.00 |
| HARDTAILS 1515 N IH 35                   | GEORGETOWN | 4,125.72 |    61,577.91 |


In [33]:
location_sum('City', 'KYLE').limit(3).print_table(max_column_width=60)

| Establishment                                       | City |  Tax_sum | Receipts_sum |
| --------------------------------------------------- | ---- | -------- | ------------ |
| CASA GARCIA'S MEXICAN RESTAURA 5401 FM 1626 STE 300 | KYLE | 4,481.69 |    66,890.90 |
| EVO ENTERTAINMENT CENTER 3200 KYLE XING             | KYLE | 3,700.34 |    55,228.96 |
| CENTERFIELD SPORTS BAR & GRILL 200 W CENTER ST      | KYLE | 2,481.34 |    37,034.93 |


In [34]:
location_sum('City', 'LAGO VISTA').limit(3).print_table(max_column_width=60)

| Establishment                                        | City       |  Tax_sum | Receipts_sum |
| ---------------------------------------------------- | ---------- | -------- | ------------ |
| COPPERHEAD GRILL 6115 LOHMANS FORD RD                | LAGO VISTA | 1,150.25 |    17,167.91 |
| MARIA'S BAR & GRILL MEXICAN RE 20602 FM 1431 STE 102 | LAGO VISTA |   400.05 |     5,970.90 |
| NATURE'S POINT LTD 18206 LAKESHORE POINT BLVD        | LAGO VISTA |     0.00 |         0.00 |


In [35]:
location_sum('City', 'LAKEWAY').limit(3).print_table(max_column_width=60)

| Establishment                                        | City    |  Tax_sum | Receipts_sum |
| ---------------------------------------------------- | ------- | -------- | ------------ |
| THE GROVE WINE BAR AND KITCHEN 3001 RANCH ROAD 620 S | LAKEWAY | 6,376.79 |    95,175.97 |
| LAKEWAY RESORT AND SPA 101 LAKEWAY DR                | LAKEWAY | 5,644.21 |    84,241.94 |
| HIGH 5 ENTERTAINMENT 1502 RANCH ROAD 620 S           | LAKEWAY | 3,940.80 |    58,817.91 |


In [36]:
location_sum('City', 'LEANDER').limit(3).print_table(max_column_width=60)

| Establishment                                        | City    |  Tax_sum | Receipts_sum |
| ---------------------------------------------------- | ------- | -------- | ------------ |
| BROOKLYN HEIGHTS PIZZERIA 3550 LAKELINE BLVD STE 135 | LEANDER | 3,345.84 |    49,937.91 |
| JARDIN DEL REY 703 S HIGHWAY 183                     | LEANDER | 2,296.82 |    34,280.90 |
| TAPATIA JALISCO #3 LLC 651 N US 183                  | LEANDER |   697.93 |    10,416.87 |


In [37]:
location_sum('City', 'LIBERTY HILL').limit(3).print_table(max_column_width=60)

| Establishment                                   | City         |  Tax_sum | Receipts_sum |
| ----------------------------------------------- | ------------ | -------- | ------------ |
| JARDIN CORONA 15395 W STATE HIGHWAY 29          | LIBERTY HILL | 2,643.88 |    39,460.90 |
| MARGARITA'S RESTAURANT 10280 W STATE HIGHWAY 29 | LIBERTY HILL | 1,745.21 |    26,047.91 |
| FIRE OAK DISTILLERY 4600 COUNTY ROAD 207        | LIBERTY HILL |     0.00 |         0.00 |


In [38]:
location_sum('City', 'PFLUGERVILLE').limit(3).print_table(max_column_width=60)

| Establishment                             | City         |  Tax_sum | Receipts_sum |
| ----------------------------------------- | ------------ | -------- | ------------ |
| MAVERICKS 1700 GRAND AVENUE PKWY STE 2    | PFLUGERVILLE | 8,554.82 |   127,683.88 |
| LAST CALL 1615 GRAND AVENUE PKWY STE 2    | PFLUGERVILLE | 3,957.62 |    59,068.96 |
| WAGNOR BROTHERS 15505 INTERSTATE 35 STE C | PFLUGERVILLE | 3,683.86 |    54,982.99 |


In [39]:
location_sum('City', 'ROUND ROCK').limit(5).print_table(max_column_width=60)

| Establishment                                | City       |  Tax_sum | Receipts_sum |
| -------------------------------------------- | ---------- | -------- | ------------ |
| JACK ALLEN'S KITCHEN 2500 HOPPE TRL          | ROUND ROCK | 8,504.24 |   126,928.96 |
| TWIN PEAKS RESTAURANT 100 LOUIS HENNA BLVD   | ROUND ROCK | 8,482.93 |   126,610.90 |
| RICK'S CABARET 3105 S INTERSTATE 35          | ROUND ROCK | 8,407.29 |   125,481.94 |
| FAST EDDIE'S NEIGHBORHOOD BILL 100 PARKER DR | ROUND ROCK | 7,910.75 |   118,070.90 |
| SALT TRADERS 2850 N INTERSTATE 35            | ROUND ROCK | 7,894.40 |   117,826.87 |


In [40]:
location_sum('City', 'SAN MARCOS').limit(5).print_table(max_column_width=60)

| Establishment                                       | City       |  Tax_sum | Receipts_sum |
| --------------------------------------------------- | ---------- | -------- | ------------ |
| THE MARC 120 E SAN ANTONIO ST                       | SAN MARCOS | 9,731.95 |   145,252.99 |
| ZELICKS 336 W HOPKINS ST                            | SAN MARCOS | 8,003.01 |   119,447.91 |
| PLUCKERS WING BAR 105 N INTERSTATE 35               | SAN MARCOS | 6,545.02 |    97,686.87 |
| 54TH STREET RESTAURANT & DRAFT 1303 S INTERSTATE 35 | SAN MARCOS | 6,093.51 |    90,947.91 |
| CHIMY'S SAN MARCOS 217 E HOPKINS ST                 | SAN MARCOS | 6,088.22 |    90,868.96 |


In [41]:
location_sum('City', 'SPICEWOOD').limit(3).print_table(max_column_width=60)

| Establishment                                      | City      |  Tax_sum | Receipts_sum |
| -------------------------------------------------- | --------- | -------- | ------------ |
| ANGEL'S ICEHOUSE 21815 W HWY 71                    | SPICEWOOD | 3,114.16 |    46,480.00 |
| POODIES HILLTOP ROADHOUSE 22308 STATE HIGHWAY 71 W | SPICEWOOD | 2,998.65 |    44,755.97 |
| APIS RESTAURANT 23526 STATE HIGHWAY 71 W           | SPICEWOOD | 2,263.93 |    33,790.00 |


In [42]:
location_sum('City', 'SUNSET VALLEY').limit(3).print_table(max_column_width=60)

| Establishment                                        | City          |  Tax_sum | Receipts_sum |
| ---------------------------------------------------- | ------------- | -------- | ------------ |
| DOC'S BACKYARD 5207 BRODIE LN # 100                  | SUNSET VALLEY | 4,704.07 |    70,210.00 |
| BJ'S RESTAURANT AND BREWHOUSE 5207 BRODIE LN STE 300 | SUNSET VALLEY | 4,663.20 |    69,600.00 |
| CANE ROSSO 4715 S LAMAR BLVD STE 103                 | SUNSET VALLEY | 1,948.09 |    29,075.97 |


In [43]:
location_sum('City', 'WEST LAKE HILLS').limit(3).print_table(max_column_width=60)

| Establishment                                               | City            |  Tax_sum | Receipts_sum |
| ----------------------------------------------------------- | --------------- | -------- | ------------ |
| LUPE TORTILLA MEXICAN RESTAURA 701 S CAPITAL OF TEXAS HWY S | WEST LAKE HILLS | 5,737.34 |    85,631.94 |
| CHIPOTLE CHIPOTLE MEXICAN GRIL 3300 BEE CAVES RD STE 670    | WEST LAKE HILLS |    25.92 |       386.87 |


## County location example

In this case, we pass in the location type of 'County' and then a county name in caps to get the most sales in a particular county.

In [44]:
location_sum('County', 'TRAVIS').limit(10).print_table(max_column_width=80)

| Establishment                                      | County |   Tax_sum | Receipts_sum |
| -------------------------------------------------- | ------ | --------- | ------------ |
| WLS BEVERAGE CO 110 E 2ND ST                       | Travis | 79,804.43 | 1,191,110.90 |
| ROSE ROOM/ 77 DEGREES 11500 ROCK ROSE AVE          | Travis | 29,873.89 |   445,878.96 |
| 400 BAR/CUCARACHA/CHUPACABRA/J 400 E 6TH ST        | Travis | 28,395.13 |   423,807.91 |
| THE DOGWOOD DOMAIN 11420 ROCK ROSE AVE STE 700     | Travis | 27,372.64 |   408,546.87 |
| KUNG FU SALOON 11501 ROCK ROSE AVE STE 140         | Travis | 23,468.15 |   350,270.90 |
| BARTON CREEK COUNTRY CLUB 8212 BARTON CLUB DR      | Travis | 23,065.68 |   344,263.88 |
| TOP GOLF 2700 ESPERANZA XING                       | Travis | 22,937.51 |   342,350.90 |
| SAN JACINTO BEVERAGE COMPANY L 98 SAN JACINTO BLVD | Travis | 22,361.92 |   333,760.00 |
| ALAMO DRAFTHOUSE CINEMA 1120 S LAMAR BLVD          | Travis | 22,324.66 |   333,203.88 |